#Data Utils
## Import libraries

In [1]:
import cv2 as cv
import tensorflow as tf
import tensorflow.keras as tfk
from tensorflow.keras import utils, layers, losses, optimizers, models, Sequential, Model
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
import os
from pathlib import Path
from collections import Counter

## Functions

In [2]:

def split_data(images, labels, train_size=0.9, shuffle=True):
    '''
       Description: split data to x_train, y_train, x_valid, y_valid
       *Args: 
       images: images as a array
       labels: labels as a array

       *Returns: 
       x_train: images using in training
       x_valid: images using in validation
       y_train: labels of training images
       y_valid: labels of validation images

    '''
    # 1. Get the total size of the dataset
    size = len(images)
    # 2. Make an indices array and shuffle it, if required
    indices = np.arange(size)
    if shuffle:
        np.random.shuffle(indices)
    # 3. Get the size of training samples
    train_samples = int(size * train_size)
    # 4. Split data into training and validation sets
    x_train, y_train = images[indices[:train_samples]], labels[indices[:train_samples]]
    x_valid, y_valid = images[indices[train_samples:]], labels[indices[train_samples:]]
    return x_train, x_valid, y_train, y_valid


def preprocess(img_path, label):
    '''
       Description: is preprocessing function which will be used in creating dataset.
                     it contains of reading image, convert to grayscale image float32, resize it 
                     to desired size and transpose(because we want the time-dimension to correspond to the width of the image) 
       *Args: 
       img_path: path of image to read it
       label: labels

       *Returns:
       {"image": img, "label": label}: a dict as our model is expecting two inputs


    '''
    # Read image
    img = tf.io.read_file(img_path)
    # Decode and convert to grayscale
    img = tf.io.decode_png(img, channels=1)
    # Convert to float32 in [0, 1] range
    img = tf.image.convert_image_dtype(img, tf.float32)
    # Resize to the desired size
    img = tf.image.resize(img, [50, 200])
    # Transpose the image because we want the time-dimension to correspond to the width of the image.
    img = tf.transpose(img, perm=[1, 0, 2])
    # Map the characters in label to numbers
    label = char_to_num(tf.strings.unicode_split(label, input_encoding="UTF-8"))
    # Return a dict as our model is expecting two inputs
    return {"image": img, "label": label}


def create_dataset(x_data, y_data, batch_size):
    '''
      Description: is a utility function to create a dataset(As tensorflow.python.data.ops.dataset_ops.PrefetchDataset Class)
                    which will be prepared for training step(it contains preprocess(mapped by preprocess function), batch, prefetch)
      *Args:
      x_data: CAPTCHA images (for training and validation step)
      y_data: text of correspondent CAPTCHA images
      batch_size: batch size

      *Returns:
      dataset: As tensorflow.python.data.ops.dataset_ops.PrefetchDataset Class

    '''
    dataset = tf.data.Dataset.from_tensor_slices((x_data, y_data))
    dataset = (dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE).batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE))
    return dataset 



